In [1]:
!pip install transformers

!pip install datasets

     |████████████████████████████████| 3.8 MB 11.9 MB/s 
     |████████████████████████████████| 895 kB 31.3 MB/s 
     |████████████████████████████████| 6.5 MB 31.6 MB/s 
     |████████████████████████████████| 596 kB 38.7 MB/s 
     |████████████████████████████████| 67 kB 4.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 12.5 MB/s 
     |████████████████████████████████| 1.1 MB 46.0 MB/s 
     |████████████████████████████████| 134 kB 52.0 MB/s 
     |████████████████████████████████| 212 kB 51.5 MB/s 
     |████████████████████████████████| 127 kB 54.3 MB/s 
     |████████████████████████████████| 144 kB 46.3 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 271 kB 51.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urll

In [3]:
from google.colab import drive
import pandas as pd 


drive.mount('/content/gdrive')

Mounted at /content/gdrive


###preprocessing

In [ ]:

import pandas as pd 
df=pd.read_csv('gdrive/My Drive/thesis/data/data.csv')
df = df.drop(df[(df.classSource =='root') & (df.classTarget =='EObject')].index)
df=df.reset_index()
del df["Unnamed: 0"]
del df['index']
df = df.astype(str)


In [ ]:
dfhead=df.head(20)
dfhead

,classSource,classTarget,assocs,packageName
0,MultiLineInputElement,ScoutingElement,inherits from,elements
1,NumberDropDownElement,ScoutingElement,inherits from,elements
2,SingleLineInputElement,ScoutingElement,inherits from,elements
3,WeightingSliderElement,ScoutingElement,inherits from,elements
4,CommentTable,H2Table,inherits from,h2
5,DictTable,H2Table,inherits from,h2
6,MatchScoutingTable,H2Table,inherits from,h2
7,WeightingTable,H2Table,inherits from,h2
8,AreaManager,StructureManager,structureManager,Manager
9,AreaManager,AreaDatabaseManager,databaseManager,Manager


In [ ]:
dataframe = pd.DataFrame([], columns = ['sequence', 'package'])
package=df.iloc[0]['packageName']
seq=[]
for i, row in df.iterrows():
   
    if row['packageName']==package:
      seq.append('('+ row['classSource'] +','+ row['assocs'] +','+ row['classTarget'] +')')
      
    else: 
      dataframe=dataframe.append({'sequence': seq, 'package': package}, ignore_index=True)
      package=df.iloc[i]['packageName']
      seq=[]
      seq.append('('+ row['classSource'] +','+ row['assocs'] +','+ row['classTarget'] +')')

dataframe

,sequence,package
0,"[(MultiLineInputElement,inherits from,Scouting...",elements
1,"[(CommentTable,inherits from,H2Table), (DictTa...",h2
2,"[(AreaManager,structureManager,StructureManage...",Manager
3,"[(BroadcastThread,subThread,BroadcastRunningTh...",Threads
4,"[(XYQuestion,inherits from,Question)]",model
...,...,...
43909,"[(MarkerPainter,_markerRenderer,MarkerRenderer...",markers
43910,"[(BottomPanel,inherits from,GenericPanel), (Ma...",panels
43911,"[(AgendaItemListRenderer,inherits from,Contact...",renderers
43912,"[(Tile,_tileFactory,TileFactory), (TileFactory...",tiles


###Data

In [ ]:
#dataframe.to_csv('gdrive/My Drive/thesis/data/data_seq.csv')

In [4]:
dataframe=pd.read_csv('gdrive/My Drive/thesis/data_seq.csv')
dataframe

,Unnamed: 0,sequence,package
0,0,"['(MultiLineInputElement,inherits from,Scoutin...",elements
1,1,"['(CommentTable,inherits from,H2Table)', '(Dic...",h2
2,2,"['(AreaManager,structureManager,StructureManag...",Manager
3,3,"['(BroadcastThread,subThread,BroadcastRunningT...",Threads
4,4,"['(XYQuestion,inherits from,Question)']",model
...,...,...,...
43909,43909,"['(MarkerPainter,_markerRenderer,MarkerRendere...",markers
43910,43910,"['(BottomPanel,inherits from,GenericPanel)', '...",panels
43911,43911,"['(AgendaItemListRenderer,inherits from,Contac...",renderers
43912,43912,"['(Tile,_tileFactory,TileFactory)', '(TileFact...",tiles


In [ ]:
'''del dataframe['package']
del dataframe['Unnamed: 0']

dataframe.sample(10000).to_csv("gdrive/My Drive/thesis/trainTest_.csv",index=False)
dataframe.tail(1000).to_csv("gdrive/My Drive/thesis/testTest_.csv",index=False)'''

In [11]:
from sklearn.model_selection import train_test_split
del dataframe['package']
del dataframe['Unnamed: 0']
train, test = train_test_split(dataframe, test_size=0.3)
train.to_csv('gdrive/My Drive/thesis/train_.csv',index=False)
test.to_csv('gdrive/My Drive/thesis/test_.csv',index=False)

In [ ]:
test=pd.read_csv('gdrive/My Drive/thesis/data/test_.csv')
test

###distilgpt2

In [ ]:
from datasets import load_dataset

#dataset = load_dataset("csv",data_files={"train":"gdrive/My Drive/thesis/trainTest_.csv", "test":"gdrive/My Drive/thesis/testTest_.csv"})
dataset

In [12]:
from datasets import load_dataset

dataset = load_dataset("csv",data_files={"train":"gdrive/My Drive/thesis/train_.csv", "test":"gdrive/My Drive/thesis/test_.csv"})
dataset

Using custom data configuration default-2a4f51bed27095e7


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-2a4f51bed27095e7/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sequence'],
        num_rows: 30739
    })
    test: Dataset({
        features: ['sequence'],
        num_rows: 13175
    })
})

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained("distilgpt2")


Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/336M [00:00<?, ?B/s]

In [14]:
from transformers import pipeline

generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

In [15]:

def tokenize_function(examples):
    return tokenizer(examples["sequence"],padding=True, truncation=True, max_length=128)

In [16]:
tokenized_dataset = dataset.map(tokenize_function, batched=True,load_from_cache_file=False)

  0%|          | 0/31 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

In [17]:

tokenized_dataset['train'] = tokenized_dataset['train'].add_column("labels", tokenized_dataset['train']['input_ids'])

tokenized_dataset['test'] = tokenized_dataset['test'].add_column("labels", tokenized_dataset['test']['input_ids'])
tokenized_dataset


DatasetDict({
    train: Dataset({
        features: ['sequence', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 30739
    })
    test: Dataset({
        features: ['sequence', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 13175
    })
})

In [18]:
import torch 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
from transformers import TrainingArguments, Trainer,default_data_collator


training_args = TrainingArguments(
    output_dir="gdrive/My Drive/thesis/data/results",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    do_train=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=default_data_collator,
    
)

trainer.train()
trainer.save_model('gdrive/My Drive/thesis/fineTunedModel')

The following columns in the training set  don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: sequence. If sequence are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 30739
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 4805


Step,Training Loss
500,0.818200
1000,0.695500
1500,0.669900
2000,0.662300
2500,0.650400
3000,0.650500
3500,0.637500
4000,0.640400
4500,0.636700


Saving model checkpoint to gdrive/My Drive/thesis/data/results/checkpoint-500
Configuration saved in gdrive/My Drive/thesis/data/results/checkpoint-500/config.json
Model weights saved in gdrive/My Drive/thesis/data/results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in gdrive/My Drive/thesis/data/results/checkpoint-500/tokenizer_config.json
Special tokens file saved in gdrive/My Drive/thesis/data/results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to gdrive/My Drive/thesis/data/results/checkpoint-1000
Configuration saved in gdrive/My Drive/thesis/data/results/checkpoint-1000/config.json
Model weights saved in gdrive/My Drive/thesis/data/results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in gdrive/My Drive/thesis/data/results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in gdrive/My Drive/thesis/data/results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to gdrive/My Drive/thesis/data/results/chec

In [24]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: sequence. If sequence are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 13175
  Batch size = 32


{'epoch': 5.0,
 'eval_loss': 0.6287959814071655,
 'eval_runtime': 173.4718,
 'eval_samples_per_second': 75.949,
 'eval_steps_per_second': 2.375}

In [23]:
trainer.save_model("gdrive/My Drive/thesis/SeqGeneration")

Saving model checkpoint to gdrive/My Drive/thesis/SeqGeneration
Configuration saved in gdrive/My Drive/thesis/SeqGeneration/config.json
Model weights saved in gdrive/My Drive/thesis/SeqGeneration/pytorch_model.bin
tokenizer config file saved in gdrive/My Drive/thesis/SeqGeneration/tokenizer_config.json
Special tokens file saved in gdrive/My Drive/thesis/SeqGeneration/special_tokens_map.json


In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token

TrainedModel = AutoModelForCausalLM.from_pretrained("gdrive/My Drive/thesis/SeqGeneration")


Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [10]:
test="(employe, worksIn,) "
#TrainedModel.to(device)
inputs = tokenizer(test)


In [11]:
from transformers import pipeline

fill_mask = pipeline(
    "text-generation",
    model=TrainedModel,
    tokenizer=tokenizer
)

In [42]:
fill_mask('(employee, worksIn,company)\',\'(business,company,Company)')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "(employee, worksIn,company)','(business,company,Company)', '(home,workin,company)', '(work,inherits from,work)']"}]